# 字母层次上的语言模型——龙之岛(Dinosaurus land)

欢迎来到“龙之岛”(Dinosaurus Island)。6500万年钱，恐龙灭绝了，如今在这里，我们即将让其重生......由生物学家主导，创造除了新品种的恐龙，而你的任务就是为这些新品种的恐龙命名（你知道的，程序员最讨厌的就是命名了）。假如你给的新名字这些恐龙们不喜欢，哼哼......就有你好果子吃了，所以要努力起好听的名字。

我们不是学了深度学习的一些东西嘛，今天刚好排上用场！
任务很简单，这里研究人员已经帮你收集了他们可以找到用来当作恐龙的名字列表以供选择，并把它放在[【这里】](https://github.com/liuqidev/deeplearning.ai.chinese/blob/master/Course%205%20Sequence%20Models/Week%201/Programming%20Assignments/dinos.txt)）。为了创造新的恐龙名字，需要构建一个字母层级上的语言模型。算法会学习到不同名字的模式，然后随机生成新的名字。希望你寄以希望的算法起的名字能是你免于恐龙之口，哈哈哈。

通过本次练习你可以学到:
- 如何使用一个循环神经网络来存储用来处理的文本数据
- 如何合成数据，这里通过在每个时间步上的采样预测以及将结果传递给洗一个RNN基本单元的方式来完成
- 如何构建一个字母层级的文本生成循环神经网络
- 知道为什么进行梯度剪裁很重要

上一篇我们已经实现了的`rnn_forward`和`rnn_backward`等函数到这里都可以直接拿来用。

In [3]:
import numpy as np
from utils import *
import random


## 1- 问题表述
### 1.1 - 数据集和处理
从恐龙的名字列表(数据集)来读取这些恐龙的名字，创建一个字母列表(比方说字母a-z)。

In [16]:
data = open('dinos.txt', 'r').read()
data = data.lower()
# print(type(data), len(data))
# print(set(data))
print(data[:20]) # 是包含回车符的
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print("There are %d words and %d unique characters in the data."%(data_size, vocab_size))

aachenosaurus
aardon
There are 19909 words and 27 unique characters in the data.


a-z 共26个英文字母，加上回车符号("\n")，和之后处理中的<EOS>（End of sentence，句子的结束）在作用上具有异曲同工之妙,这里的话只表示名字的结束而不是整个句子的结束。由于你知道的处理字符串不是很给力，所以这里创建了一个python字典（你也可以用说它是一个哈希表）来讲每个字母映射成其对应的数字（0-26）。别担心，同样创建了另外一个python字典，将数字映射回来成为对应的字母。这将协助你弄清楚在之后经过softmax分类器输出的概率密度分布中什么数字缩影对应这什么字母。

In [18]:
char_to_ix = {ch:i for i, ch in enumerate(sorted(chars))} # 字母映射成为数字索引
ix_to_char= {i:ch for i, ch in enumerate(sorted(chars))} # 数字索引映射成为字母
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### 1.2 - 模型总体

我们需要的模型结构总体长成下面这样：
- 初始化的各种参数
- 进行优化训话过程：
    - 前向传播，计算loss函数
    - 反向传播，计算关于loss的梯度
    - 梯度裁剪来防止梯度爆炸
    - 根据提到下降原则使用计算得到的梯度来更新参数
- 返回学习到的参数

在每个时间步，RNN都试图通过给定的之前的字母来预测下一个字母将会是啥。训练集$X= (x^{\langle 1\rangle}, x^{\langle 2\rangle},..., x^{\langle T_{x}\rangle})$是一个字母列表。$Y= (y^{\langle 1 \rangle}, y^{\langle 2 \rangle},...,y^{\langle T_{x} \rangle})$，其中每个也都是字母，注意：$y^{\langle t \rangle}=x^{\langle t+1 \rangle}$,也就是前一个预测的结果作为了下一个时间步的输入。

## 2 - 为模型添砖加瓦

上面我们已经设计好了整个模型的总体结构，接下来先实现其最重要的两个构件：
- 梯度裁剪（Gradient Clipping）：为了避免梯度爆照
- 采样（sampling）：核心科技，用来生成字母。

只有先实现了这两个，才能逐渐构建起整个模型。

### 3.1 - 在优化的循环过程中进行梯裁剪

即将实现的`clip`函数是用来子优化的循环中进行调用的。通常整个大循环中包含，前向传播，损失(cost)计算，反向传播，以及参数更行这几个流程。在更行参数之前，需要对梯度进行裁剪，防止梯度太大而“爆炸”。

梯度裁剪并非只有一种方式，我们使用简单的逐个元素梯度裁剪这样一种简单的方式，就是将每个计算得到的梯度的范围先订到$[-N, N]$的范围上，更一般的，找一个最大值maxValue(设成10吧)，举个例子，如果梯度向量中哪个项值超过了10，就把它设成10；如果哪项小于-10了，就把它设置成-10。如果值在[-10, 10]之间，那就保持。

**实战**：实现梯度裁剪函数，返回字典`gradients`裁剪后的梯度。函数接收一个最大的阈值，返回裁剪后的梯度，返回的是经过裁剪的梯度。具体实现由困难的话，这里有[【小抄】](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html)。你会使用声明`out=....`的。

In [39]:
### 实现函数，clip

def clip(gradients, maxValue):
    """
    将gradients（梯度）裁剪到最小值和最大值之间
    
    参数：
    gradients -- python 字典包含梯度"dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- 上述的值超过maxValue 就设置成它；小于-maxValue就是设置成-maxValue
    
    返回：
    gradients -- 裁剪之后的字典
    """
    
    dWaa, dWax, dWya, db, dby = gradients["dWaa"], gradients["dWax"], gradients["dWya"],gradients["db"],gradients["dby"]
    
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
            
    gradients = {"dWaa": dWaa, "dWax":dWax, "dWya":dWya, "db":db, "dby":dby}
    
    return gradients

In [40]:
np.random.seed(3)
dWax = np.random.randn(5, 3)*10
dWaa = np.random.randn(5, 5)*10
dWya = np.random.randn(2, 5)*10
db = np.random.randn(5, 1)*10
dby = np.random.randn(2, 1)*10
gradients = { "dWax":dWax, "dWaa": dWaa, "dWya":dWya, "db":db, "dby":dby}
gradients = clip(gradients, 10)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 10.]
gradients["dby"][1] = [ 8.45833407]


### 2.2 0 - 采样Sampling
现在假设你的模型已经被训练了。你现在想让模型生成下一个文本（字母）。整个的处理过程如下图所示：

**实战**：实现`sample`函数来对字母进行采样，需要执行四步：
- 第一步：象征性地将第一个输入输送个网络$x^{\langle 1 \rangle}=\vec{0} $(全零向量)。这是用来生成任何字母的默认输入，对了，$a^{\langle 0 \rangle}=\vec{0}$，也是设置成零向量的哟。

- 第二步：向前迈出第一步，也就是进行第一步前向传播，从而得到$a^{\langle 1 \rangle}$以及$y^{\langle 1 \rangle}$。公式如下：
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

注意，$\hat{y}^{t+1}$是一个softmax概率向量(也就是其所有项都是介于0和1之间的概率值，并且所有相加等于1)。$\hat{y}_{i}^{t+1}$ 代表着索引为"i"的字母是下一个字母的概率。
- 第三步：开始采样：根据$\hat{y}^{t+1}$的概率分布从而选择下一个字母的索引，比方说，$\hat{y}_i^{t+1}=0.16$,那么需要选择的索引为i的字母的概率为16%。具体的实现中，你可能需要用到[`np.random.choice`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)


**np.random.choice()**的使用例子：

In [42]:
np.random.seed(0)
p = np.array([0.1, 0.0, 0.7, 0.2])
index = np.random.choice([0, 1, 2, 3],p=p.ravel())
print(p)
print(index)

[ 0.1  0.   0.7  0.2]
2


上面的例子表明，我们根据概率分布选取`index`：
$P(index=0)=0.1, P(index=1)=0.0, P(index=2)=0.7, P(index=3)=0.2$

- 第四步：实现`sample()`的最后一步用$x^{\langle t+1 \rangle}$来覆盖掉当前存储着$x^{\langle t \rangle}$变量`x`的值。$x^{\langle t+1 \rangle}$使用一位热码的方式来表示。接着就继续向前传播，知道遇到字符'\n'停止，也就是说你已经为你的恐龙起好名字了。

In [45]:
# 实现函数: sample

def sample(parammeters, char_to_ix, seed):
    """
    根据一个概率分布的序列，采样得到字母序列。
    
    参数：
    parameters -- python字典，包含参数 Waa, Wax, Wya, by 和b
    char_to_ix -- python字典，将每个字母映射成索引
    seed -- 
    
    返回：
    indices -- 长度为n的列表，包含已经采样的字母的索引
    
    """
    # 从变量"parameters"中取回参数
    Waa, Wax, Wya, by, b = parammeters["Waa"], parammeters["Wax"], parammeters["Wya"],parammeters["by"], parammeters["b"]
    vocab_size=by.shape[0]
    n_a = Waa.shape[1]
    
    # 第一步，对第一个字母创建一个one-hot 向量x（用来初始化整个序列）
    x = None
    # 初始化a_prep为全零向量
    a_prev = None
    
    # 创建一个空的索引列表，这个列表将包含要生成的字母的索引
    indices = []
    
    # Idx是一个用来检测是否有新行字母的标志，将其初始化为-1
    idx = -1
    
    # 接下来是整个时间序列上的循环，在每个时间不上，从概率分布上采样一个字母
    # 并将其索引添加到索引列表indexs中。假如采样到50个字符，我们将会停止（这
    # 对一个训练较好的模型几乎是不可能发生的），这一步能够帮助我们避免陷入死循环。
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # 步骤2：前向传播，运用上述公式，（1）（2）（3）
        a = None 
        z = None
        y = None 
        
        # 
        np.random.seed(counter+=seed)
        
        # 步骤3 , 从y的概率分布中得到一个字母的索引
        idx = None
        
        # 将index添加到indexs中
        None
        
        # 步骤4， 
        x = None
        x[None] = None
        
        # 
        a_prev = None
        
        # 
        seed+= 1
        counter += 1
        
    if (counter==50):
        indices.append(char_to_ix['\n'])
    
    return indices
    
    

SyntaxError: invalid syntax (<ipython-input-45-f4cce0c51b9e>, line 46)

In [ ]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:", indices)
print("list of sampled characters:", [ix_to_char[i] for i in indices])